In [2]:
#Tarea Dask: Carlos Castro Correa 103531
#Adolfo: Primero voy a acomodar y procesar un poco mi base de datos, posteriormente voy a responder las preguntas de 
# tarea "obligatoria" señalando claramente, pregunta y respuesta.

#Exportamos delayed
from dask import delayed
#Para leer el archivo csv como dataframe
from dask import dataframe
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


In [3]:
# Truco para convertir a datetime...
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime.head()

0   2015-01-03 01:37:02
1   2015-01-05 23:35:02
2   2015-01-06 15:22:12
3   2015-01-08 08:31:23
4   2015-01-08 12:35:54
Name: tpep_dropoff_datetime, dtype: datetime64[ns]

In [4]:
#Voy a tomar la columna de duración como variable X en mi predicción bajo el supuesto de que "entre más lejos es un viaje, más propina deja"
#podrías pensar en agregar duración como variable pero me parece que está muy realcionada al tiempo total
# Agrega una columna para la duración del viaje
trips_df["trip_duration"] = trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime
trips_df.trip_duration.map(lambda d: d.total_seconds()).head()

0    1170.0
1     587.0
2     627.0
3     551.0
4     568.0
Name: trip_duration, dtype: float64

In [5]:
#podemos observar la columna ya agregada, como trip_duration en segundos
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28


In [6]:
#Creamos una columna con la proporción de propina de acuerdo al viaje
trips_df["prop"] = trips_df.tip_amount / trips_df.fare_amount
#Observamos la columna de propoción ya creada
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration,prop
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,0.209091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,0.133333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,0.117647
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,0.221333


In [7]:
#Primero voy a hacer mi ejercicio con los dataframe de Pandas, posteriormente voy a probar con Dask-ML
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [8]:
df = pd.read_csv("/data/trips.csv")
df = pd.DataFrame(df)
#Proporcion de la propina
df['prop'] =  df.tip_amount / df.fare_amount

#Me quedo con la parte del dataframe que voy a utilizar en mi regresion
df = df[['prop','trip_distance']]
df = df.dropna(axis=0, how='any')#Quitamos cualquier NA de nuestros registros para no tener problemas al ajustar el modelo
df[0:10]

,prop,trip_distance
0,0.209091,6.90
1,0.000000,1.81
2,0.133333,0.96
3,0.117647,1.90
4,0.221333,1.00
5,0.157895,1.71
6,0.187500,1.27
7,0.213333,1.55
8,0.260000,0.54
9,0.576923,15.38


In [9]:
y = df.prop
#Separo mis datos en entrenamiento y prueba, utilizaré el 20%
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
#Tamaño de mi conjunto de prueba
X_train.shape, y_train.shape

((7356, 2), (7356,))

In [10]:
#Tamaño de mi conjunto de prueba
X_test.shape, y_test.shape

((1839, 2), (1839,))

In [15]:
#Ajustamos un modelo de regresión lineal
lm = linear_model.LinearRegression(fit_intercept=False)
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)
#Mostamos el valor de las primeras 10 predicciones
predictions[0:10]


array([3.12331192e-18, 2.10666667e-01, 3.74797430e-18, 2.77333333e-01,
       2.32500000e-01, 7.06140086e-18, 1.40956425e-17, 8.41936257e-18,
       2.90000000e-01, 3.39490426e-18])

In [23]:
#Podemos observar que hay 3 parámetros con los que podemos probar: intercept, normalize, n_jobs
lm.fit

<bound method LinearRegression.fit of LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)>

In [46]:

##Tarea - aquí respondo las preguntas

#A continuación, voy a realizar las corridas en cada una de las 3 opciones y posteriormente,
#agergo un párrafo con las respuestas a las prguntas.
import numpy as np

% time
#Busqueda secuencial desde Python
from sklearn.grid_search import GridSearchCV
model = linear_model.LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[10,20,30]}
grid = GridSearchCV(model,parameters, cv=3)
grid.fit(X_train, y_train)
"r2 / variance : ", grid.best_score_
grid.best_score_
"Residual sum of squares: %.2f"% np.mean((grid.predict(X_test) - y_test) ** 2)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16.9 µs


'Residual sum of squares: 0.00'

In [37]:
"Mejores parámetros secuencial : ", grid.best_params_

('Mejores parámetros secuencial : ',
 {'copy_X': True, 'fit_intercept': True, 'n_jobs': 10, 'normalize': True})

In [61]:
#Búsqueda con Dask-ML
%time
from dask.distributed import Client
client = Client()  # start a local Dask client

import dask_ml.joblib
from sklearn.externals.joblib import parallel_backend
with parallel_backend('dask'):
    from sklearn.grid_search import GridSearchCV
    model = linear_model.LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[10,20,30]}
    grid_dask = GridSearchCV(model,parameters, cv=3)
    grid_dask.fit(X_train, y_train)
    "r2 / variance : ", grid_dask.best_score_
    grid_dask.best_score_
    "Residual sum of squares: %.2f"% np.mean((grid_dask.predict(X_test) - y_test) ** 2)
    grid_dask.best_params_

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 29.1 µs


In [189]:
"Mejores parámetros con Dask : ", grid_dask.best_params_

('Mejores parámetros con Dask : ',
 {'copy_X': True, 'fit_intercept': True, 'n_jobs': 10, 'normalize': True})

In [60]:
#Búsqueda con la función delay
% time
def paralelo(A):
    from sklearn.grid_search import GridSearchCV
    model = linear_model.LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[10,20,30]}
    grid = GridSearchCV(model,parameters, cv=3)
    grid.fit(X_train, y_train)
    return grid.best_params_

delayed_grid = delayed(grid)
print("Paralelo:")


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.5 µs
Paralelo:


In [52]:
#Respuesta a las preguntas de tarea.

#* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python? 

#Podemos observar que correr en secuencial tardó 16.9 ms mientras que en paralelo (con delay) se tardó 15.5 ms;
#observamos que a pesar de que esta base de datos no es tan grande, el algoritmo en paralelo corrió más rápido.
#En algunos casos, para pocos datos, los códigos en secuencial tardan menos (porque el paralelo debe repartir tareas)
#en este caso, no resultó así y ganó el uso de la función delay.

## * ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML? 

#El uso de Dask ML no mejoró el tiempo de procesamiento, de hecho, fue el que más se tardó (21.1 ms), seguramente está relacionado
#con el tamaño de la base datos, tal vez, con un conjunto más grande funcionaría más rápido pero no fue el caso.